In [24]:
using LinearAlgebra

### Scalar Neural Network

In [25]:
h(x) = exp(-x)
h′(x,y) = -y
𝓁(x,y) = sum(abs2,x-y)/2
𝓁′(x,y) = x-y
𝜀 = .0001

0.0001

In [26]:
N = 3 
w = randn(N)
b = randn(N)
params =[[w[i],b[i]] for i=1:N]

3-element Array{Array{Float64,1},1}:
 [0.409429, 1.63166]  
 [1.5489, -0.610416]  
 [0.0139285, -1.40057]

In [27]:
function neural_net(params, input; h=h, h′=h′, N=length(params))
    δ = [];
    X = [input];
    for i=1:N
        x = params[i]⋅[X[i],1]
        y = h.(x) 
        push!(δ, h′.(x,y))
        push!(X,y)
    end
    return X,δ
end

neural_net (generic function with 1 method)

In [28]:
x = rand()
y = 0.5
X,δ   = neural_net(params,x)
L     = Bidiagonal(zeros(N),δ[2:N].*w[2:N],:L)
D     = Diagonal(δ.*[[X[i],1]' for i=1:N])
f     = [zeros(N-1);𝓁′(X[N+1],y)]
∇J    = D'*((I-L')\f)

3-element Array{Array{Float64,1},1}:
 [-0.017232, -0.0742665]
 [0.0479479, 0.269554]  
 [-19.3528, -13.8452]   

In [29]:
∇Jfd = ∇J * 0
ϵ    = ∇J * 0
for i=1:N, j=1:2       
    ϵ[i][j] = 𝜀
    ∇Jfd[i][j]=(𝓁(neural_net(params.+ϵ,x)[1][N+1],y)-𝓁(neural_net(params.-ϵ,x)[1][N+1],y))/2𝜀
    ϵ[i][j] = .0
end
∇Jfd

3-element Array{Array{Float64,1},1}:
 [-0.017232, -0.0742665]
 [0.0479479, 0.269554]  
 [-19.3528, -13.8452]   

### Simple Matrix Neural Network
- [] needed to create a box type
- [] There are some issues with adjoint, did we assume transpose recursive in the article? If it is below code works accordingly.
- [] However, adjoint definitions for ⊗, ⊗′ problematic.

In [30]:
import Base: .+,.*,.-,+,-,*,zero,one,adjoint,inv,/,./,convert,size,isequal,iszero,getindex, setindex!

abstract type Op; end
struct (⊗) <: Op; A; B; end
struct ⊗′ <: Op; A; B; end
struct Δ  <: Op; A; end

-(K::⊗′) = -K.A ⊗′ K.B 
*(K::⊗′,X::Union{AbstractArray,Number}) = K.B * (K.A * X) # changing paranthesis gives wrong result
adjoint(K::⊗′) = K.B ⊗′ K.A' # same issue with kronocker adjoint

-(K::⊗) = -K.A ⊗ K.B
*(K::⊗,X::Union{AbstractArray,Number}) = (K.B * X) * K.A' # changing paranthesis gives dimension errors
adjoint(K::⊗) = K.A' ⊗ K.B  # not consistent with adjoint of kroncker, if we keep consistent
                            # then the elements of D becomes X[j]' ⊗ Δ(δ[i])', not consistent with article

-(X::Δ) = Δ(-X.A)
*(X::Δ,Y::Union{AbstractArray,Number}) = X.A .* Y
*(Y::Union{AbstractArray,Number},X::Δ) = Y .* X.A
adjoint(X::Δ) = Δ(X.A')

# I can't think another way than boxing elements 
#to handle with zeros and ones requires by the backslash and triangular matrices
struct Box; K; end
# Unary Definitions
zero(::Type{Box}) = Box(0)
one(::Type{Box}) = Box(1)
zero(::Box) = zero(Box)
one(::Box) = one(Box)
value(R::Box) = R.K
iszero(R::Box) = R.K==0
adjoint(R::Box) = Box(adjoint(R.K))
inv(R::Box) = Box(inv(R.K))
#Binary Definitions
convert(::Type{Box},x::Union{Number,V,T}) where V <: Op where T <: AbstractArray = Box(x)
/(X::Number,R::Box) = Box(X*inv(R))
/(R1::Box,R2::Box) = Box(R1*inv(R2))
-(R::Box) = Box(-R.K)
-(R::Box, X::AbstractArray) = Box(R.K-X)
-(X::AbstractArray,R::Box)  = Box(X-R.K)
-(R1::Box,R2::Box) = Box(R1.K.-R2.K)
+(R1::Box,R2::Box) = Box(R1.K.+R2.K)
*(R1::Box,R2::Box) = Box(R1.K*R2.K)
*(R::Box,X::Union{Number,V,T}) where V <: Op where T <: AbstractArray = Box(R.K * X)
*(X::Union{Number,V,T},R::Box) where V <: Op where T <: AbstractArray = Box(X * R.K)

* (generic function with 411 methods)

In [31]:
#Needed to overwrite naivesub!, see comment in 12th line
import LinearAlgebra: naivesub!, has_offset_axes
function naivesub!(A::UnitUpperTriangular, b::AbstractVector, x::AbstractVector = b)
    @assert !has_offset_axes(A, b, x)
    n = size(A, 2)
    if !(n == length(b) == length(x))
        throw(DimensionMismatch("second dimension of left hand side A, $n, length of output x, $(length(x)), and length of right hand side b, $(length(b)), must be equal"))
    end
    @inbounds for j in n:-1:1
        xj = x[j] = b[j]
        for i in j-1:-1:1
            if !iszero(A.data[i,j]) # EKIN: added this line to solve zero shape related problem. 
                                    # Otherwise, in matrix neural network we get dimension mismastch
                b[i] -= A.data[i,j] * xj
            end
        end
    end
    x
end

array(x) = fill(x,1,1)

array (generic function with 1 method)

In [32]:
function neural_net(params,input;h=h,h′= h′)
    X     = [input]
    δ     = []
    for i=1:length(params)
        x = params[i][1]*X[i] .+ params[i][2]         
        push!(X,h.(x))
        push!(δ,h′.(x,X[i+1]))
    end 
    X,δ
end

neural_net (generic function with 1 method)

In [33]:
n = [3 3 3 1]
N = length(n)-1
B = 7
W = [randn(n[i+1],n[i]) for i=1:N]
b = [randn(n[i+1]) for i=1:N]
params =[[W[i],b[i]] for i=1:N]

3-element Array{Array{Array{Float64,N} where N,1},1}:
 [[-0.364712 0.144268 1.23095; -0.89484 -0.399452 -0.991036; 0.573454 -0.491818 -0.732605], [-0.275151, 0.433567, -0.21384]]
 [[0.833144 -0.64884 -0.713167; 1.30563 -0.551628 1.20049; 0.78175 -0.393685 0.156015], [-1.20916, 0.699874, 1.36389]]      
 [[-0.348164 -0.393493 0.507356], [0.0681898]]                                                                              

In [34]:
y = randn(1,B)*0.1
x = randn(n[1],B)*0.1
X,δ = neural_net(params,x)
D = Diagonal([[X[i]' ⊗  Δ(δ[i]) Δ(δ[i]')] for i=1:N])
L = Bidiagonal(fill(Box(0),N), [Box(params[i][1] ⊗′ Δ(δ[i])) for i=2:N] , :L)
f = [Box(0) for i=1:N]
f[N] = 𝓁′(X[N+1],y)
∇J = D'*array.((UnitUpperTriangular(-L')\f))
∇Jw = value.(first.(∇J))
∇Jb = value.(getindex.(∇J,2))

3-element Array{Array{Float64,2},1}:
 [8.36974 11.4023 … 12.966 90.7926; -2.86048 -4.14977 … -4.47754 -46.5517; -4.67462 -7.267 … -9.03337 -105.503]            
 [-6.4221 -9.27072 … -10.9274 -105.357; -0.0658502 -0.0854999 … -0.0927914 -0.675302; 0.265256 0.355902 … 0.403374 3.04383]
 [-6.37543 -8.05009 … -9.04041 -50.748]                                                                                    

In [35]:
∇Jfd = params*0
ϵ=params*0
for i=1:length(params), wb=1:2
    for j=1:length(ϵ[i][wb])
            ϵ[i][wb][j] = 𝜀
            ∇Jfd[i][wb][j] =(𝓁(neural_net(params+ϵ,x)[1][N+1],y)-𝓁(neural_net(params-ϵ,x)[1][N+1],y))/2𝜀
            ϵ[i][wb][j] = .0
     end
end
∇Jfd

3-element Array{Array{Array{Float64,N} where N,1},1}:
 [[-13.2795 -5.12737 9.84107; 5.61082 2.92235 -6.03449; 15.0273 6.62905 -13.764], [184.322, -77.7411, -180.31]]     
 [[-225.501 -121.406 -254.018; -1.58558 -0.838219 -1.73198; 7.19316 3.79558 7.89745], [-188.942, -1.30391, 5.92884]]
 [[-542.681 -3.31367 -11.6858], [-114.064]]                                                                         

In [36]:
∇Jfd[1][1]

3×3 Array{Float64,2}:
 -13.2795   -5.12737    9.84107
   5.61082   2.92235   -6.03449
  15.0273    6.62905  -13.764  

In [37]:
∇Jw[1]

3×3 Array{Float64,2}:
 -13.2795   -5.12737    9.84107
   5.61082   2.92235   -6.03449
  15.0273    6.62905  -13.764  

In [38]:
∇Jfd[1][2]

3-element Array{Float64,1}:
  184.32190001085758
  -77.74109494530279
 -180.30987600941017

In [50]:
sum(∇Jb[1];dims=2) # biasses needs to be sum up in batch dimension, since they are broadcasted.

3×1 Array{Float64,2}:
  184.3218969749331 
  -77.7410934801986 
 -180.30986442829192

### Densely Connected Matrix Network

 - [ ] Defined step matrices to make D'* possible with generic zeros.

In [40]:
"""
Horizontal Step Matrix
    [---
        ---
           ---]

Vertical Step Matrix
    [|
     |
       |
       |
       |
         |]
"""
abstract type StepMatrix{T} <: AbstractMatrix{T}; end;
size(S::StepMatrix) = (S.m, S.n)

struct VerticalStepMatrix{T} <: StepMatrix{T}
    m::Int
    n::Int
    colptr::Vector{Int}
    values::Vector{T}    
    function VerticalStepMatrix{T}(m::Int, n::Int, colptr::Vector{Int},values::Vector{T}) where T
        m < 0 && error("rows m $m")
        n < 0 && error("rows n $n")
        new(Int(m), Int(n), colptr, values)
    end
end

getindex(S::VerticalStepMatrix{T},i::Integer, j::Integer) where T = (S.colptr[i] == j ? values[i] : zero(T))
setindex!(S::VerticalStepMatrix{T},value::T,i::Integer,j::Integer) where T = (S.colptr[i] = j; values[i] = value)
VerticalStepMatrix(t::Type{T},m::Int,n::Int) where T  = VerticalStepMatrix{T}(m, n, Vector{Int}(undef,m),Vector{T}(undef,m))

struct HorizontalStepMatrix{T} <: StepMatrix{T}
    m::Int
    n::Int
    rowptr::Vector{Int}
    values::Vector{T}    
    function HorizontalStepMatrix{T}(m::Int, n::Int, colptr::Vector{Int},values::Vector{T}) where T
        m < 0 && error("rows m $m")
        n < 0 && error("rows n $n")
        new(Int(m), Int(n), colptr, values)
    end
end
getindex(S::HorizontalStepMatrix{T},i::Integer, j::Integer) where T = (S.rowptr[j] == i ? S.values[j] : zero(T))
setindex!(S::HorizontalStepMatrix{T},value::T, i::Integer, j::Integer) where T = (S.rowptr[j] = i; S.values[j] = value)
HorizontalStepMatrix(t::Type{T},m::Integer, n::Integer) where T  = HorizontalStepMatrix{T}(m, n, Vector{Int}(undef,n),Vector{T}(undef,n))

adjoint(S::VerticalStepMatrix{T}) where T = HorizontalStepMatrix{T}(S.n,S.m, S.colptr, adjoint.(S.values))
adjoint(S::HorizontalStepMatrix{T}) where T = VerticalStepMatrix{T}(S.n,S.m, S.rowptr, adjoint.(S.values))

function (*)(S::VerticalStepMatrix{T}, X::AbstractVector) where T
    results = Vector{T}(undef,S.m)
    for i=1:S.m
        results[i] = S.values[i] * X[S.colptr[i]]
    end
    return results
end

* (generic function with 411 methods)

In [41]:
function neural_net(params,input;h=h,h′= h′)
    X     = [input]
    δ     = []
    layeroffset = 0; i = 1
    while layeroffset < length(params)
       x = zeros(n[i+1],B)
       for j=1:i
           x += params[layeroffset+j]*X[j]  
       end    
       push!(X,h.(x .+ params[layeroffset+i+1]))
       push!(δ,h′.(x,X[i+1]))
       i+=1; layeroffset+=i
    end 
    X,δ
end

neural_net (generic function with 1 method)

In [42]:
n = [2 3 2 1]
N = length(n)-1
B = 7
params = []
for i=1:N
    wi = []
    for j=1:i
        push!(params,randn(n[i+1],n[j]))
    end
    push!(params,randn(n[i+1]))
end

In [43]:
y = randn(1,B)*0.1
x = randn(n[1],B)*0.1
X,δ = neural_net(params,x)
D = HorizontalStepMatrix(Box,N,sum(2:N+1))
L = LowerTriangular(zeros(Box,N,N))
for i=1:N
    layeroffset = sum(2:i)
    for j=1:i
        D[i,layeroffset+j] = Box(X[j]' ⊗ Δ(δ[i]))
        if i>1 && j!=i
            L[i,j] = Box(params[layeroffset+j+1] ⊗′ Δ(δ[i]))
        end
    end
    D[i,layeroffset+i+1] = Box(Δ(δ[i]'))
end

f = [Box(0) for i=1:N] # [zeros(n[i+1],B) for i=1:N] may use this without modifying naivesub!. However,matrix neural network won't work 
f[N] = 𝓁′(X[N+1],y)

∇J=D'*(UnitUpperTriangular(-L')\f) # Trick: I-L' = UnitUpperTriangular(-L')
∇J=value.(∇J)

9-element Array{Array{Float64,2},1}:
 [6.55357e-7 -5.60905e-7; 5.52284e-6 -3.8049e-6; -3.02292e-7 2.33001e-7]                                                                        
 [-1.4427e-8 2.67441e-6 … 5.84388e-9 1.22249e-6; -1.18827e-7 1.9039e-5 … 5.60568e-8 7.0659e-6; 5.96991e-9 -1.15829e-6 … -2.36938e-9 -4.57883e-7]
 [-1.4041e-7 1.05284e-7; 5.60857e-6 -4.30496e-6]                                                                                                
 [-1.58873e-6 -8.97747e-7 -4.38173e-7; 6.42688e-5 3.61923e-5 1.76933e-5]                                                                        
 [3.70509e-9 -5.04579e-7 … -1.92665e-9 -2.19364e-7; -1.28588e-7 2.09442e-5 … 5.89779e-8 8.79205e-6]                                             
 [-4.52938e-7 3.33163e-7]                                                                                                                       
 [-5.12828e-6 -2.91306e-6 -1.41792e-6]                                                       

In [44]:
∇Jfd = params*0
ϵ=params*0
for i=1:length(params)
    for j=1:length(ϵ[i])
            ϵ[i][j] = 𝜀
            ∇Jfd[i][j] =(𝓁(neural_net(params+ϵ,x)[1][N+1],y)-𝓁(neural_net(params-ϵ,x)[1][N+1],y))/2𝜀
            ϵ[i][j] = .0
     end
end
∇Jfd

9-element Array{Array{Float64,N} where N,1}:
 [6.55357e-7 -5.60905e-7; 5.52284e-6 -3.8049e-6; -3.02292e-7 2.33e-7]   
 [5.77296e-6, 4.60926e-5, -2.59625e-6]                                  
 [-1.4041e-7 1.05284e-7; 5.60857e-6 -4.30496e-6]                        
 [-1.58873e-6 -8.97747e-7 -4.38173e-7; 6.42688e-5 3.61923e-5 1.76933e-5]
 [-1.186e-6, 4.79291e-5]                                                
 [-4.52938e-7 3.33163e-7]                                               
 [-5.12828e-6 -2.91306e-6 -1.41792e-6]                                  
 [-1.05468e-6 -1.85894e-5]                                              
 [-3.84028e-6]                                                          

In [45]:
∇J[1]

3×2 Array{Float64,2}:
  6.55357e-7  -5.60905e-7
  5.52284e-6  -3.8049e-6 
 -3.02292e-7   2.33001e-7

In [46]:
∇Jfd[1]

3×2 Array{Float64,2}:
  6.55357e-7  -5.60905e-7
  5.52284e-6  -3.8049e-6 
 -3.02292e-7   2.33e-7   

In [47]:
∇J[2] # biasses needs to be sum up in batch dimension, since they are broadcasted.

3×7 Array{Float64,2}:
 -1.4427e-8    2.67441e-6  -4.91386e-8  …   5.84388e-9   1.22249e-6
 -1.18827e-7   1.9039e-5   -5.46281e-7      5.60568e-8   7.0659e-6 
  5.96991e-9  -1.15829e-6   2.04782e-8     -2.36938e-9  -4.57883e-7

In [48]:
∇J[2] = sum(∇J[2];dims=2)

3×1 Array{Float64,2}:
  5.772956965034854e-6
  4.609262547142157e-5
 -2.59624744148595e-6 

In [49]:
∇Jfd[2]

3-element Array{Float64,1}:
  5.772956904182003e-6 
  4.609263155180843e-5 
 -2.5962474531349145e-6